# Test Env

Import data.

In [1]:
%load_ext autoreload
%autoreload 2

import dgym as dg

# load all data
print('load data')
path = '../dgym-data'

deck = dg.MoleculeCollection.from_sdf(
    f'{path}/DSi-Poised_Library_annotated.sdf',
    reactant_names=['reagsmi1', 'reagsmi2', 'reagsmi3']
)

reactions = dg.ReactionCollection.from_json(
    path = f'{path}/All_Rxns_rxn_library.json',
    smarts_col = 'reaction_string',
    classes_col = 'functional_groups'
)

load data


In [2]:
building_blocks = dg.datasets.enamine(f'{path}/Enamine_Building_Blocks_Stock_262336cmpd_20230630.sdf')
fingerprints = dg.datasets.fingerprints(f'{path}/out/Enamine_Building_Blocks_Stock_262336cmpd_20230630.fpb')

# align fingerprints to building blocks
print('align fingerprints')
fingerprints = dg.utils.sort_fingerprints(fingerprints, building_blocks)

# partition building blocks according to functional groups
print('partition building blocks')
templates = dg.utils.get_unique_reactants(reactions)
building_blocks = dg.utils.partition_building_blocks(building_blocks, templates, out_dir=path)

align fingerprints
partition building blocks


In [3]:
from dgym.envs.library_designer import LibraryDesigner

library_designer = LibraryDesigner(
    reactions,
    building_blocks,
    fingerprints
)

In [4]:
from dgym.envs.oracle import DGLOracle

lipo_oracle = DGLOracle('GCN_canonical_Lipophilicity')
esol_oracle = DGLOracle('GCN_canonical_ESOL')
# esol_oracle.predict(deck)

In [5]:
from dgym.envs import DrugEnv

drug_env = DrugEnv(
    library_designer,
    library=deck[:200],
    assays=[lipo_oracle],
    budget=200
)
# is there data associated with the library? could be.

action = {
    'design': {
        'molecules': [0, 50],
        'num_analogs': 2,
        'fraction_random': 0.0
        # fraction_random
    }
}

drug_env.step(action)
# [display(p) for p in drug_env.library[6:]]

(MoleculeCollection with 212 Molecules, None, None, {})

In [6]:
drug_env = DrugEnv(
    library_designer,
    library = deck[:10],
    assays = [lipo_oracle],
    budget = 10
)

action = drug_env.action_space.sample()
action['design']['molecules'] = list(set(action['design']['molecules']))
action['order']['molecules'] = list(set(action['order']['molecules']))

drug_env.step(action)

(MoleculeCollection with 50 Molecules, None, None, {})

In [7]:
molecules = drug_env.library[action['order']['molecules']]

In [21]:
molecules[0].annotations

{'assay_0': 2.3975718021392822}

In [37]:
rdkit.Chem.Draw.IPythonConsole.ipython_maxProperties=-1
dg.molecule.Molecule(molecules[0].mol)

NameError: name 'rdkit' is not defined

In [13]:
drug_env.library[3].annotations

{'assay_0': -0.2128268927335739}

In [32]:
from dgym.agents import DrugAgent

In [33]:
drug_agent = DrugAgent(drug_env.action_space)